<a href="https://githubtocolab.com/jmvazqueznicolas/sistemas_inteligentes_22-1/blob/master/P6_Sistemas%20Inteligentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

## Práctica 6. Sistemas Inteligentes

### Objetivos
#### 1.- Que el alumno implemente una red neuronal básica.
### Marco teórico
#### El alumno debe investigar lo que son las redes neuronales.

### Desarrollo

### 1.-  Crea un programa que permita reconocer digitos escritos a mano, usando un subconjunto de la base de datos [MNIST](https://interactivechaos.com/es/manual/tutorial-de-deep-learning/el-dataset-mnist). Utiliza una red neuronal básica para este programa.
<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [ ]:
num_classes = 10
num_features = 784 # Tamaño de los datos (28x28=784) 

# Parametros del entrenamiento de la red neuronal (Hiperparamétros)
learning_rate = 0.1
epocas = 10
batch_size = 256
display_step = 50

# Parametros de la red neuronal
n_hidden_1 = 128
n_hidden_2 = 256
n_hidden_3 = 512
n_hidden_4 = 512
n_hidden_5 = 256
n_hidden_6 = 256
n_hidden_7 = 256
n_hidden_8 = 128
n_hidden_9 = 128
n_hidden_10 = 128

In [ ]:
# Descargar y preparar MNIST
from tensorflow.keras.datasets import mnist 
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("En numero de datos de entrenamiento", len(y_train))
print("En numero de datos de prueba", len(y_test))
print("La forma de los datos es",x_train[0].shape)


# Convertir datos 
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
print("La forma de la primer imagen antes del aplanado", x_train.shape)
# Aplanando
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
print("La forma de la primer imagen despues del aplanado", x_train.shape)
# Normalizacion
x_train, x_test = x_train/255., x_test/255.
print("El primer dato despues de la normalización", x_train[0])

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 784), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>


In [ ]:
# Clase para crear la red neuronal
class NeuralNet(Model):
  # Creando capas
  def __init__(self):
    super(NeuralNet, self).__init__()
    # Primer capa densa (fuertemente conectada (todos-todos))
    self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
    self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
    self.fc3 = layers.Dense(n_hidden_3, activation=tf.nn.relu)
    self.fc4 = layers.Dense(n_hidden_4, activation=tf.nn.relu)
    self.fc5 = layers.Dense(n_hidden_5, activation=tf.nn.relu)
    self.fc6 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
    self.fc7 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
    self.fc8 = layers.Dense(n_hidden_3, activation=tf.nn.relu)
    self.fc9 = layers.Dense(n_hidden_4, activation=tf.nn.relu)
    self.fc10 = layers.Dense(n_hidden_5, activation=tf.nn.relu)
    self.out = layers.Dense(num_classes)
  
  def call(self, x, is_training=False):
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.fc3(x)
    x = self.fc4(x)
    x = self.fc5(x)
    x = self.fc6(x)
    x = self.fc7(x)
    x = self.fc8(x)
    x = self.fc9(x)
    x = self.fc10(x)
    x = self.out(x)
    if not is_training:
      x = tf.nn.softmax(x)
    return x

# Se crea la red neuronal
neural_net = NeuralNet()

In [ ]:
# Funcion de pérdida
def cross_entropy_loss(x, y):
  y = tf.cast(y, tf.int64)
  loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
  return tf.reduce_mean(loss)

# Métricas
def accuracy(y_pred, y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Optimizador
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
def run_optimization(x, y):
  with tf.GradientTape() as g:
    pred = neural_net(x, is_training=True)
    # Calculando la función de pérdida
    loss = cross_entropy_loss(pred, y)
  # Paramétros de la red (pesos sinapticos)
  trainable_variables = neural_net.trainable_variables
  # Cálculo del gradiente
  gradients = g.gradient(loss, trainable_variables)
  optimizer.apply_gradients(zip(gradients, trainable_variables))



In [ ]:
# Parametros del entrenamiento de la red neuronal (Hiperparamétros)
learning_rate = 0.1
epocas = 6000
batch_size = 256
display_step = 100
neural_net = NeuralNet()

# Inicio del entrenamiento
for step, (batch_x, batch_y) in enumerate(train_data.take(epocas), 1):
  run_optimization(batch_x, batch_y)
  if step % display_step == 0:
    pred = neural_net(batch_x, is_training=True)
    loss = cross_entropy_loss(pred, batch_y)
    acc = accuracy(pred, batch_y)
    print(f"epoca:{step}, loss:{loss}, acc:{acc}")

epoca:100, loss:1.72971773147583, acc:0.49609375
epoca:200, loss:0.9617166519165039, acc:0.6328125
epoca:300, loss:0.21734768152236938, acc:0.94140625
epoca:400, loss:0.1984747350215912, acc:0.93359375
epoca:500, loss:0.15656304359436035, acc:0.953125
epoca:600, loss:0.11126787960529327, acc:0.97265625
epoca:700, loss:0.10882722586393356, acc:0.96484375
epoca:800, loss:0.07810398191213608, acc:0.984375
epoca:900, loss:0.08489019423723221, acc:0.9765625
epoca:1000, loss:0.07886683940887451, acc:0.9765625
epoca:1100, loss:0.04815538972616196, acc:0.98828125
epoca:1200, loss:0.046705685555934906, acc:0.984375
epoca:1300, loss:0.031038636341691017, acc:0.9921875
epoca:1400, loss:0.020464330911636353, acc:0.9921875
epoca:1500, loss:0.04357253015041351, acc:0.984375
epoca:1600, loss:0.033529412001371384, acc:0.99609375
epoca:1700, loss:0.007553417235612869, acc:1.0
epoca:1800, loss:0.054128848016262054, acc:0.98828125
epoca:1900, loss:0.026309635490179062, acc:0.9921875
epoca:2000, loss:0.02

In [ ]:
# Probando el modelo en el conjunto de prueba
pred = neural_net(x_test, is_training=False)
print(x_test.shape)
print(f"Acc en prueba: {accuracy(pred, y_test)}")

(10000, 784)
Acc en prueba: 0.9781000018119812


In [ ]:
neural_net.summary()
print("pesos que llegan a la primer capa", (784*8) + 8)

print("pesos que llegan a la segunda capa", (8 * 16) + 16)

print("pesos que llegan a la capa de salida", (16*10) + 10)

In [ ]:
import matplotlib.pyplot as plt
n_images = 100
test_images = x_test[:n_images]
predictions = neural_net(test_images)

# Mostrando las imagenes y su prediccion
for i in range(n_images):
  plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
  plt.show()
  print(f"Prediccion del modelo {np.argmax(predictions.numpy()[i])}")